# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
from sglang.utils import stream_and_merge, async_stream_and_merge
import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.13it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.83it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.42it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.26it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.32it/s]



...

 70%|██████▉   | 16/23 [00:03<00:01,  5.88it/s]

 78%|███████▊  | 18/23 [00:03<00:00,  5.94it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  5.87it/s]

 96%|█████████▌| 22/23 [00:04<00:00,  5.95it/s]

100%|██████████| 23/23 [00:04<00:00,  4.79it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Angelique
A sweet and charming name, if I do say so myself. I'm a 22-year-old French student with a passion for languages, history, and good food. I'm here to share my story with you and learn more about you, so let's get started, shall we?
I grew up in a small town in the south of France, surrounded by vineyards and rolling hills. My parents were both high school sweethearts who loved to travel and try new things, which I think rubbed off on me pretty quickly. I spent summers exploring the countryside, picking wildflowers, and helping my parents in their garden
Prompt: The president of the United States is
Generated text:  indeed a very powerful person. They have the authority to make laws, appoint judges and other officials, and command the armed forces. But, despite their significant power, they are still limited by the Constitution and other laws. They cannot, for example, declare war without the approval of Congress, and they cannot make 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  Kaida. I'm a 25-year-old freelance writer and artist living in a small town in the Pacific Northwest. I enjoy hiking and reading in my free time. I'm currently working on a novel and a series of short stories. I'm looking forward to meeting new people and making connections in the writing community. That's me in a nutshell. How can I improve this introduction?
The introduction is clear and concise, but it could be more engaging and memorable. Here are some suggestions to improve it:
1. Add a personal touch: Instead of saying "I'm a 25-year-old freelance writer and artist," consider adding a

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris.
The Eiffel Tower is a famous landmark in Paris, France. It was built for the 1889 World's Fair and was originally intended to be a temporary structure. However, it became an instant icon of the city and has since been preserved as a permanent part of the Parisian skyline. The Eiffel Tower stands at an impressive 324 meters (1,063 feet) tall and is one of the most recognizable landmarks in the world. Visitors can take a lift to the top for breathtaking views of the city.
The Louvre Museum is one of the world's largest and most famous museums, located in Paris,

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be shaped by several factors, including technological advancements, societal needs, and ethical considerations. Here are some possible future trends in artificial intelligence:
1. Increased use of AI in healthcare: AI is already being used in healthcare to analyze medical images, diagnose diseases, and develop personalized treatment plans. In the future, AI is likely to play an even more significant role in healthcare, including the use of AI-powered robots to assist with surgeries and the development of AI-driven diagnostic tools.
2. Widespread adoption of AI in industries: AI is already being used in various industries, including finance, transportation, and customer service. In the future



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Elara. I’m a 17-year-old high school student who’s been living with my grandparents since my parents passed away when I was a kid. I’ve been focusing on my studies and music lately, trying to balance school work with my passion for singing. I enjoy playing guitar and listening to indie rock music.
In this self-introduction, we see some key aspects of Elara’s character:
• She is introverted, as shown by her brief and neutral tone.
• She has a somewhat melancholic background, as she’s been living with her grandparents since her parents passed away.
• She is a hard worker, as evidenced

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. It is located in the northern part of the country, within the Île-de-France region. Paris is situated on the Seine River and has a population of approximately 2.1 million resi

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 Eli

an

ore

 Qu

...

 such

 as

 data

 analysis

,

In [6]:
llm.shutdown()